# Classification modeling - Restaurants



## Import modules and data

In [1]:
import pandas as pd
import numpy as np

## Load the review metadata set `rests` and the doc-topic matrix `lsa_matrix`

In [2]:
rests = pd.read_csv('../data/restaurants.csv', compression='gzip')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
dt_matrix = np.load('../data/lsa_matrix.npy')

FileNotFoundError: [Errno 2] No such file or directory: '../data/lsa_matrix.npy'

In [ ]:
rests.shape, dt_matrix.shape

To correctly combine our metadata and topic vectors into one feature matrix, we need to ensure they are the same shape:

In [2]:
correct_index = np.load('../data/rests_eng_index.npy')
rests = rests[rests.index.isin(correct_index)]

In [ ]:
rests.reset_index(drop=True)

In [ ]:
rests.shape, dt_matrix.shape

## Drop null values

In [ ]:
rests[rests.isnull().any(axis=1)]

In [ ]:
right_matrix = np.delete(dt_matrix, [], 0)
right_matrix.shape

## Create validation set following usefulness schema

In [25]:
def useful_mapper(x):
    if x == 0:
        return 0
    elif x in (1, 2):
        return "Validation"
    elif x >= 3:
        return 1

In [26]:
rests['Usefulness'] = rests['useful'].map(useful_mapper)

In [ ]:
rests['Usefulness'].value_counts()

In [ ]:
rests.shape

## Split out validation and train topic vectors

In [ ]:
valid_index = bus[bus['usefulness'] == 'Validation'].index

In [ ]:
right_matrix_valid = right_matrix[valid_index]
right_matrix_valid.shape

In [ ]:
right_matrix_train = np.delete(right_matrix, valid_index, 0)
right_matrix_train.shape

## Create validation and train metadata datasets and save to csv

In [ ]:
rests_valid = rests[rests.index.isin(valid_index)]

rests_valid.to_csv('../data/rests_validation.csv', index=False)

In [ ]:
rests_train = rests[~rests.index.isin(valid_index)]

rests_train.to_csv('../data/rests_train.csv', index=False)

In [ ]:
rests_train[0].shape, rests_valid[0].shape

## Create train feature matrix

In [ ]:
rests_train.drop(['text', 'useful', 'cool', 'state'], 1, inplace=True)

left_matrix_train = rests_train[rests_train.columns[:-1]].values

In [ ]:
left_matrix_train.shape, right_matrix_train.shape

del bus, bus_train, correct_index, dt_matrix, right_matrix

features = np.hstack((left_matrix_train, right_matrix_train))

## Create validation feature matrix

In [ ]:
rests_valid = pd.read_csv('../data/rests_validation.csv')

rests_valid.columns

rests_valid.drop(['text', 'useful', 'cool', 'state'], 1, inplace=True)

left_matrix_valid = rests_valid[rests_valid.columns[:-1]].values

In [ ]:
left_matrix_valid.shape, right_matrix_valid.shape

In [ ]:
del bus, bus_valid, correct_index, dt_matrix, right_matrix

valid_features = np.hstack((left_matrix_valid, right_matrix_valid))

np.save('../data/valid_features_rests.npy', valid_features)

## Create train target vector

In [ ]:
rests_train = pd.read_csv('../data/rests_train.csv')

In [ ]:
np.save('../data/rests_target.npy', rests_train.usefulness.values)